In [1]:
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
from collections import Counter
#import gensim
import jieba
import json
import pickle
import csv
import numpy as np
#from gensim.scripts.glove2word2vec import glove2word2vec

np.random.seed(0)
random.seed(0)

In [2]:
# load user video act
json_file = r"C:\Users\user\Desktop\Heterogeneous\MOOCCube\additional_information\user_video_act.json"

with open(json_file, "r") as f:
    lines = f.readlines()

# load course name dict
with open(r"C:\Users\user\Desktop\Heterogeneous\MOOCCube\entities\course.json", "r", encoding="utf8") as f:
    courses = f.readlines()
    
course_dict = dict()
for c in courses:
    json_data = json.loads(c)
    course_dict[json_data["id"]] = json_data["name"]

# load course-teacher relationship to dict
def get_course_dict(filepath):
    with open(filepath, "r", encoding="utf8") as f:
        tc_rels = f.readlines()
    tc_rel_dict = dict()
    for tc_rel in tc_rels:
        tc_rel = tc_rel.strip().split("\t")
        if tc_rel[1] not in tc_rel_dict:
            tc_rel_dict[tc_rel[1]] = [tc_rel[0]]
        else:
            tc_rel_dict[tc_rel[1]].append(tc_rel[0])
        
    return tc_rel_dict

tc_rel_dict = get_course_dict(r"C:\Users\user\Desktop\Heterogeneous\MOOCCube\relations\teacher-course.json")
print("total courses", len(tc_rel_dict))

sc_rel_dict = get_course_dict(r"C:\Users\user\Desktop\Heterogeneous\MOOCCube\relations\school-course.json")
print("total courses", len(sc_rel_dict))

# load video concepts
vc_df = pd.read_csv(r"C:\Users\user\Desktop\Heterogeneous\MOOCCube\relations\video-concept.json",
                   header=None,
                   delimiter="\t",
                   names=["video", "concept"])
vc_dict = vc_df.groupby('video')['concept'].apply(list).to_dict()

# load course concepts
cc_df = pd.read_csv(r"C:\Users\user\Desktop\Heterogeneous\MOOCCube\relations\course-concept.json",
                   header=None,
                   delimiter="\t",
                   names=["course", "concept"])
cc_dict = cc_df.groupby('course')['concept'].apply(list).to_dict()

# load course video
cv_df = pd.read_csv(r"C:\Users\user\Desktop\Heterogeneous\MOOCCube\relations\course-video.json",
                   header=None,
                   delimiter="\t",
                   names=["course", "video"])
cv_dict = cv_df.groupby("course")["video"].apply(list).to_dict()

total courses 697
total courses 705


In [3]:
print("total users", len(lines))
user_dfs = list()

for l in tqdm(lines):
    json_data = json.loads(l)
#     print(json_data)

    activities = json_data["activity"]
    course_ids = list()
    video_ids = list()
    local_start_times = list()
    concepts = list()
    cnames = list()
    teachers = list()
    schools = list()
    
    for a in activities:
        cnames.append(course_dict[a["course_id"]]) if a["course_id"] in course_dict else cnames.append("")
        course_ids.append(a["course_id"])
        video_ids.append(a["video_id"])
        local_start_times.append(a["local_start_time"])
        concepts.append(vc_dict[a["video_id"]]) if a["video_id"] in vc_dict else concepts.append([""])
        teachers.append(tc_rel_dict[a["course_id"]]) if a["course_id"] in tc_rel_dict else teachers.append([""])
        schools.append(sc_rel_dict[a["course_id"]][0]) if a["course_id"] in sc_rel_dict else schools.append("")
        
    df = pd.DataFrame({
        "id": json_data["id"],
        "cname": cnames,
        "cid": course_ids,
        "vid": video_ids,
        "concepts": concepts,
        "teachers": teachers,
        "schools": schools,
        "local_start_time": local_start_times
    })
    df.sort_values("local_start_time", inplace=True)
    
#     display(df)
    user_dfs.append(df)

# convert to total df
total_df = pd.concat(user_dfs,ignore_index=True)
total_df.shape

total users 48640


100%|██████████| 48640/48640 [00:24<00:00, 1982.94it/s]


(4874298, 8)

In [4]:
# max '2020-04-17 17:25:48'
bad_formatted_users = list(total_df[total_df["local_start_time"]<"2017-01-01 00:00:00"]['id'].unique())
user_in_trainingperiod = list(total_df[(total_df["local_start_time"]>='2017-01-01 00:00:00') & (total_df["local_start_time"]<='2019-11-01 00:00:00')]['id'].unique())
user_in_testperiod = list(total_df[total_df["local_start_time"]>'2019-11-01 00:00:00']['id'].unique())
selected_users = [u for u in user_in_testperiod if u in user_in_trainingperiod if u not in bad_formatted_users]
print(len(selected_users))

filtered_df = total_df[total_df["id"].isin(selected_users)]
print(filtered_df.shape)

# check whether there is at least one concept in the testing period that is not in the training period
need_to_filter = list()
for u in tqdm(selected_users):
    udf = filtered_df[filtered_df["id"]==u]
    train_udf = udf[(udf["local_start_time"]>='2017-01-01 00:00:00') & (udf["local_start_time"]<='2019-11-01 00:00:00')]
    test_udf = udf[udf["local_start_time"]>'2019-11-01 00:00:00']
    train_concepts = list(set([x for sublist in train_udf["concepts"].values for x in sublist]))
    test_concepts = list(set([x for sublist in test_udf["concepts"].values for x in sublist]))
    if len([x for x in test_concepts if x not in train_concepts])==0:
#         print("unsatisfied user:", u)
        need_to_filter.append(u)

filtered_users = [u for u in selected_users if u not in need_to_filter]
filtered_df = filtered_df[filtered_df["id"].isin(filtered_users)]
print(filtered_df.shape)

2130
(311743, 8)


100%|██████████| 2130/2130 [00:19<00:00, 109.21it/s]

(290024, 8)


In [5]:
test_df= total_df[total_df["local_start_time"]>'2019-11-01 00:00:00']

In [6]:
print("---------------- Entity Statistics -----------------")
distinct_concepts = list(set([c for sublist in filtered_df["concepts"].values for c in sublist if len(c)>0]))
#test_distinct_concepts = list(set([c for sublist in test_df["concepts"].values for c in sublist if len(c)>0]))
print("filtered df", filtered_df.shape)
print("total users", len(filtered_df["id"].unique()))
print("total courses", len(filtered_df["cid"].unique()))
print("total videos", len(filtered_df["vid"].unique()))
print("total schools", len(filtered_df["schools"].unique()))
print("distinct concepts", len(distinct_concepts))
t_list = filtered_df["teachers"].values
flattend_t_list = list(set([t for sublist in t_list for t in sublist]))
print("total teachers", len(flattend_t_list))


---------------- Entity Statistics -----------------
filtered df (290024, 8)
total users 2005
total courses 600
total videos 22403
total schools 137
distinct concepts 21037
total teachers 1385


In [7]:
# # store distinct concept for embedding analysis
# with open(r"concept21037.txt", "w") as f:
#     for c in distinct_concepts:
#         f.write("{}\n".format(c))

In [8]:
print("---------------- Relation Statistics based on filtered courses -----------------")
print("user-course relations", filtered_df.groupby(['id','cid']).size().shape[0])

# relations based on filtered course 
courses = filtered_df["cid"].unique()
# course video
filtered_cv_dict = dict(filter(lambda elem: elem[0] in courses, cv_dict.items()))
print("course-video relations", sum([len(x) for x in filtered_cv_dict.values()]))
# teacher course
filtered_tc_rel_dict = dict(filter(lambda elem: elem[0] in courses, tc_rel_dict.items()))
print("teacher-course relations", sum([len(x) for x in filtered_tc_rel_dict.values()]))
# video concept
videos_in_filtered_courses = list(set([x for sublist in filtered_cv_dict.values() for x in sublist]))
print(len(videos_in_filtered_courses))
filtered_vc_rel_dict = dict(filter(lambda elem: elem[0] in videos_in_filtered_courses, vc_dict.items()))
print("video-concept relations", sum([len(x) for x in filtered_vc_rel_dict.values()]))
# course concept
filtered_cc_rel_dict = dict(filter(lambda elem: elem[0] in courses, cc_dict.items()))
print("course-concept relations", sum([len(x) for x in filtered_cc_rel_dict.values()]))
# store KC.p
concepts = distinct_concepts
course2index = dict(zip(courses, list(range(len(courses)))))
concept2index = dict(zip(concepts, list(range(len(concepts)))))

cc_list = list()
for c in courses:
    cvec = np.zeros(len(concepts))
    if c in filtered_cc_rel_dict:
        indices = [concept2index[_] for _ in filtered_cc_rel_dict[c] if _ in concept2index]
        cvec[indices] = 1
    else:
        print("course - {} does not have concepts".format(c))
    cc_list.append(cvec)
    
cc_np = np.array(cc_list).T
cc_np.shape

---------------- Relation Statistics based on filtered courses -----------------
user-course relations 13696
course-video relations 42117
teacher-course relations 1875
34506
video-concept relations 295475
course-concept relations 150811
course - C_course-v1:RiceX+Phys102x+sp does not have concepts
course - C_course-v1:TsinghuaX+Thesis2018+sp does not have concepts
course - C_course-v1:UC_BerkeleyX+ColWri2_1x_2015_T1+2019_T1 does not have concepts
course - C_course-v1:HUBU+2017022703X+sp does not have concepts
course - C_course-v1:TsinghuaX+TsinghuaMandarin01+sp does not have concepts
course - C_course-v1:UC_BerkeleyX+CS169_2x+sp does not have concepts
course - C_course-v1:ZAFU+20171218+2019_T1 does not have concepts
course - C_course-v1:qhnu+20181212x+2019_T1 does not have concepts
course - C_course-v1:SDSNAssociation+C21001+sp does not have concepts
course - C_course-v1:XJTU+20171025001+2019_T1 does not have concepts
course - C_course-v1:NUDT+05028103+2018_T2 does not have concepts
co

(21037, 600)

In [9]:
def get_ua(udf, column, index_dict):

    if isinstance(udf[column].values[0], list):
        uconcepts = list([c for sublist in udf[column].values for c in sublist if len(c)>0])
    else:
#         print(udf[column].values)
        uconcepts = list(c for c in udf[column].values if len(c)>0)
    uconcepts = Counter(uconcepts)
    uconcepts = dict(uconcepts)
#     print(len(uconcepts))
    
    uconcepts_indices = [index_dict[c] for c in uconcepts.keys()]
    uvec = np.zeros(len(index_dict))
#     uvec[uconcepts_indices] = 1
    np.put(a=uvec, ind=uconcepts_indices, v=list(uconcepts.values()))
    assert len(uconcepts_indices) == (uvec>0).sum()
    return uvec

def radom_negative_sample(user_action, item_size):
    """to get (user_size, 100, 2), 100th item is positive one"""
    negative_sample = []
    for u in user_action:
        sample = []
        i = 0
        while i < 99:
            t = random.randint(0, item_size-1)
            if t not in user_action[u]:
                sample.append([u, t])
                i += 1
        sample.append([u, user_action[u][-1]])
        negative_sample.append(sample)
    return np.array(negative_sample)

In [10]:
# creat user-index, concept-index dict
users = filtered_df["id"].unique()
user2index = dict(zip(users, list(range(len(users)))))
concepts = distinct_concepts
concept2index = dict(zip(concepts, list(range(len(concepts)))))
courses = filtered_df["cid"].unique()
course2index = dict(zip(courses, list(range(len(courses)))))
teachers = flattend_t_list
teacher2index = dict(zip(teachers, list(range(len(teachers)))))
schools = filtered_df["schools"].unique()
school2index = dict(zip(schools, list(range(len(schools)))))
videos = filtered_df["vid"].unique()
video2index = dict(zip(videos, list(range(len(videos)))))


user_action = list()
rating_matrix = list()
adjacency_matrix = list()
user_course = list()
user_teacher = list()
user_school = list()
user_video = list()
user_concept_dict = dict()

time_filtered_df = filtered_df[(filtered_df["local_start_time"]>='2017-01-01 00:00:00') & (filtered_df["local_start_time"]<='2019-11-01 00:00:00')]

for i in tqdm(range(len(users))):
    # user_action
    udf = filtered_df[filtered_df.id==users[i]]    
    udf_train = udf[(udf["local_start_time"]>='2017-01-01 00:00:00') & (udf["local_start_time"]<='2019-11-01 00:00:00')]
    uvec_train = get_ua(udf_train, 'concepts', concept2index)
    adjacency_matrix.append(uvec_train)
    # rating_matrix
    udf_train.sort_values(["local_start_time"], inplace=True)
    con_list = udf_train["concepts"].values
    learning_time = udf_train["local_start_time"].values
    con_dict = dict()
    for ind, clist in enumerate(con_list):
        time = learning_time[ind]
        for c in clist:
            if c not in con_dict and len(c)>0:
                con_dict[c] = time
    uvec_train_ = uvec_train.copy()
    uvec_train_[concept2index[list(con_dict.keys())[-1]]] = 0
    rating_matrix.append(uvec_train_)

    user_concept_dict[i] = [concept2index[c] for c in list(con_dict.keys())]

    user_action.append(get_ua(udf, 'concepts', concept2index))
    user_course.append(get_ua(udf_train, 'cid', course2index))
    user_school.append(get_ua(udf_train, 'schools', school2index))
    user_teacher.append(get_ua(udf_train, 'teachers', teacher2index))
    user_video.append(get_ua(udf_train, 'vid', video2index))
    
matrices = [adjacency_matrix, user_course, user_teacher, user_school, user_video]
matrices = [(np.array(matrix) > 0).astype(np.int8) for matrix in matrices]
adjacency_matrix, user_course, user_teacher, user_school, user_video = matrices


  0%|          | 0/2005 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_8036\2867057873.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  udf_train.sort_values(["local_start_time"], inplace=True)
100%|██████████| 2005/2005 [00:19<00:00, 105.26it/s]


In [11]:
# courses
course_action = list()
course_teacher = list()
course_video = list()
for i in tqdm(range(len(courses))):
    try:
        udf = filtered_df[filtered_df.cid==courses[i]]   
        udf_train = udf[(udf["local_start_time"]>='2017-01-01 00:00:00') & (udf["local_start_time"]<='2019-11-01 00:00:00')] 
        uvec_train = get_ua(udf_train, 'concepts', concept2index)
        udf_train.sort_values(["local_start_time"], inplace=True)

        course_action.append(get_ua(udf, 'concepts', concept2index))
        course_teacher.append(get_ua(udf_train, 'teachers', teacher2index))
        course_video.append(get_ua(udf_train, 'vid', video2index))
    except : 
        course_action.append(np.zeros(len(concept2index)))
        course_teacher.append(np.zeros(len(teacher2index)))
        course_video.append(np.zeros(len(video2index)))

course_action = np.vstack(course_action)
course_teacher = np.vstack(course_teacher)
course_video = np.vstack(course_video)

  0%|          | 0/600 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_8036\244062866.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  udf_train.sort_values(["local_start_time"], inplace=True)
100%|██████████| 600/600 [00:10<00:00, 54.68it/s]


In [12]:
teacher_action = list()
teacher_video = list()
df_exploded = filtered_df.explode('teachers')
for i in tqdm(range(len(teachers))):
    try:
        udf = df_exploded[df_exploded.teachers==teachers[i]]  
        udf_train = udf[(udf["local_start_time"]>='2017-01-01 00:00:00') & (udf["local_start_time"]<='2019-11-01 00:00:00')]  
        uvec_train = get_ua(udf_train, 'concepts', concept2index)
        udf_train.sort_values(["local_start_time"], inplace=True)
        
        teacher_action.append(get_ua(udf, 'concepts', concept2index))
        teacher_video.append(get_ua(udf_train, 'vid', video2index))
    except : 
        teacher_action.append(np.zeros(len(concept2index)))
        teacher_video.append(np.zeros(len(video2index)))
teacher_action = np.vstack(teacher_action)
teacher_video = np.vstack(teacher_video)

  0%|          | 0/1385 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_8036\1989059790.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  udf_train.sort_values(["local_start_time"], inplace=True)
100%|██████████| 1385/1385 [00:29<00:00, 46.66it/s]


In [13]:
video_action =  list()
for i in tqdm(range(len(videos))):
    try:
        udf = time_filtered_df[time_filtered_df.vid==videos[i]]    
        udf_train = udf[(udf["local_start_time"]>='2017-01-01 00:00:00') & (udf["local_start_time"]<='2019-11-01 00:00:00')]
        uvec_train = get_ua(udf_train, 'concepts', concept2index)
        udf_train.sort_values(["local_start_time"], inplace=True)
        video_action.append(get_ua(udf, 'concepts', concept2index))
    except : 
        video_action.append(np.zeros(len(concept2index)))
video_action = np.vstack(video_action)

100%|██████████| 22403/22403 [05:36<00:00, 66.65it/s]


In [14]:
import numpy as np

user_length = adjacency_matrix.shape[0]
kc_length =  adjacency_matrix.shape[1]
course_length =  user_course.shape[1]
teacher_length =  user_teacher.shape[1]
video_length = user_video.shape[1]

total_rows = user_length +kc_length+course_length+teacher_length+video_length

combined_matrix = np.zeros((total_rows, total_rows), dtype=np.int8)

col_0 = user_length
col_1 = col_0 + kc_length
col_2 = col_1 + course_length
col_3 = col_2 + teacher_length
col_4 = col_3 + video_length

row_0 = user_length
row_1 = row_0 + kc_length
row_2 = row_1 + course_length
row_3 = row_2 + teacher_length
row_4 = row_3 + video_length

combined_matrix[:row_0, col_0:col_1] = adjacency_matrix
combined_matrix[:row_0, col_1:col_2] = user_course
combined_matrix[:row_0, col_2:col_3] = user_teacher
combined_matrix[:row_0, col_3:col_4] = user_video

# course  
combined_matrix[row_0:row_1, col_1:col_2] = course_action.T
combined_matrix[row_2:row_3, col_1:col_2] = course_teacher.T
combined_matrix[row_3:row_4, col_1:col_2] = course_video.T

# teacher
combined_matrix[row_0:row_1, col_2:col_3] = teacher_action.T
combined_matrix[row_3:row_4, col_2:col_3] = teacher_video.T

#video 

combined_matrix[row_0:row_1, col_3:col_4] = video_action.T

print(combined_matrix.shape)
combined_matrix = combined_matrix+combined_matrix.T

(47430, 47430)


In [15]:
adjacency_matrix.T[5877-user_length].nonzero()

(array([ 397,  459,  749, 1164, 1285, 1473], dtype=int64),)

In [19]:
f =0
for i in range(50000):
    if len(combined_matrix[:,i].nonzero()[0]) == 0 :
        print(i)
      
print(f)

8205
10980
11021
16422
16695
18537
20277
22432
23291
23538
23566
23617
23841
23934
24062
24149
24194
24291
24394
24527
24590
24765
24770
24813
24943
27919
27920
27922
27923
27924
27925
27926
27927
27928
27929
27930
27931
27932
27933
27934
27935
27936
27937
27938
27940
27941
27942
27943
27944
27945
27946
27947
27948
27950
27951
27952
31519
31543
31544
32062
32063
32064
32065
32066
32069
32070
32071
32072
32073
32074
32075
32076
32077
32078
32079
32080
32081
32082
32083
32084
32085
32086
32087
32088
32121
32128
32143
32416
32417
32718
33048
34456
34656
34668
34705
34706
34707
34708
34709
34710
34872
35116
35136
35557
35562
35571
35582
35583
35586
35594
35600
35602
35603
35610
35612
35623
35624
35625
35641
36039
36059
36060
36505
36601
36846
37086
37087
37088
37103
37245
37322
37559
37560
37561
37565
37566
37572
37574
37575
37605
37766
37887
37888
37889
37890
37891
37893
37894
37895
37896
37897
37898
37981
38619
38620
38621
38622
38639
38640
38642
38645
38646
38647
38648
38649
38650
38651

IndexError: index 47430 is out of bounds for axis 1 with size 47430

In [17]:
f

866

In [ ]:
values = [adjacency_matrix.shape[0], adjacency_matrix.shape[1]]

# Calculate cumulative sum
cumulative_indices = np.cumsum(values).tolist()

print(cumulative_indices)

[2005, 23042]


In [ ]:
def radom_negative_sample(user_action, item_size):
    """to get (user_size, 100, 2), 100th item is positive one"""
    negative_sample = []
    for u in user_action:
        sample = []
        i = 0
        while i < 99:
            t = random.randint(0, item_size-1)
            if t not in user_action[u]:
                sample.append([u, t])
                i += 1
        sample.append([u, user_action[u][-1]])
        negative_sample.append(sample)
    return np.array(negative_sample)

In [ ]:
# construct negatives for the last item in each user's training set
negatives = radom_negative_sample(user_concept_dict, len(concept2index))
negatives.shape

(2005, 100, 2)

In [ ]:
(np.array(rating_matrix)>0).sum(), (np.array(adjacency_matrix)>0).sum()

(856067, 858072)

In [ ]:
(rating_matrix[0]>0).sum(), adjacency_matrix[0].sum()

(386, 387)

In [ ]:
# check shape
print("user_action", np.array(user_action).shape)
print("rating_matrix", np.array(rating_matrix).shape)
print("all_adjacency_matrix", np.array(combined_matrix).shape)
print("adjacency_matrix", np.array(adjacency_matrix).shape)
#print("UK", np.array(user_kc).shape)
print("UC", np.array(user_course).shape)
print("UCT", np.array(user_teacher).shape)
print("US", np.array(user_school).shape)
print("UV", np.array(user_video).shape)
print("negatives", negatives.shape)

user_action (2005, 21037)
rating_matrix (2005, 21037)
all_adjacency_matrix (47430, 47430)
adjacency_matrix (2005, 21037)
UC (2005, 600)
UCT (2005, 1385)
US (2005, 137)
UV (2005, 22403)
negatives (2005, 100, 2)


In [ ]:
with open(r'C:\Users\user\Desktop\Heterogeneous\kgc-rec\data/all_adjacency_matrix.p', 'wb') as f:
    pickle.dump(np.asmatrix(combined_matrix), f)

In [ ]:
with open('data/KC.p', 'wb') as f:
    pickle.dump(np.asmatrix(cc_np), f)
with open('data/teacher_concept.p', 'wb') as f:
    pickle.dump(np.asmatrix(teacher_action), f)

FileNotFoundError: [Errno 2] No such file or directory: 'data/KC.p'

In [ ]:
# map the files from paper repository
with open('data/user_action.p', 'wb') as f:
    pickle.dump(np.asmatrix(user_action), f)
with open('data/rate_matrix.p', 'wb') as f:
    pickle.dump(np.asmatrix(rating_matrix), f)
with open('data/adjacency_matrix.p', 'wb') as f:
    pickle.dump(np.asmatrix(adjacency_matrix), f)
with open('data/all_adjacency_matrix.p', 'wb') as f:
    pickle.dump(np.asmatrix(combined_matrix), f)
with open('data/UC.p', 'wb') as f:
    pickle.dump(np.asmatrix(user_course), f)
with open('data/UCT.p', 'wb') as f:
    pickle.dump(np.asmatrix(user_teacher), f)
with open('data/US.p', 'wb') as f:
    pickle.dump(np.asmatrix(user_school), f)
with open('data/UV.p', 'wb') as f:
    pickle.dump(np.asmatrix(user_video), f)
with open('data/negative.p', 'wb') as f:
    pickle.dump(negatives, f)
with open('data/teacher_concept.p', 'wb') as f:
    pickle.dump(np.asmatrix(teacher_action), f)

In [ ]:
import gensim
model = gensim.models.fasttext.load_facebook_model('data/cc.zh.300.bin')
from sklearn.preprocessing import LabelEncoder

video_concept = []  # Step 1: Initialize an empty list
for v in videos:
    vvec = np.zeros(len(concepts))
    if v in filtered_vc_rel_dict:
        indices = [concept2index[_] for _ in filtered_vc_rel_dict[v] if _ in concept2index]
        vvec[indices] = 1
    video_concept.append(vvec)
with open('data/video_concept.p', 'wb') as f:
    pickle.dump(np.asmatrix(video_concept), f)

course_concept = []  
for c in courses:
    cvec = np.zeros(len(concepts)) 
    if c in filtered_cc_rel_dict:
        indices = [concept2index[_] for _ in filtered_cc_rel_dict[c] if _ in concept2index]
        cvec[indices] = 1
    course_concept.append(cvec)
with open('data/course_concept.p', 'wb') as f:
    pickle.dump(np.asmatrix(course_concept), f)

teacher_df = pd.read_json(r"C:\Users\user\Desktop\Heterogeneous\MOOCCube\entities\teacher.json", lines=True)
sentences = teacher_df[teacher_df['id'].isin(teachers)]['about'].tolist()
sentences.insert(0, '1')
def get_sentence_embedding(sentence, model):
    embeddings = [model.wv[word] for word in sentence.split() if word in model.wv]
    if embeddings:
        return sum(embeddings) / len(embeddings)
    else:
        print(word)
        return None
        
sentence_embeddings = [get_sentence_embedding(sentence, model) for sentence in sentences]
with open('data/teacher_embedding.p', 'wb') as f:
    pickle.dump(np.array(sentence_embeddings), f)

con_vectors = list()
for c in distinct_concepts:
    if c in model.wv:  
        con_vectors.append(model.wv[c])

with open('data/concept_embedding.p', 'wb') as f:
    pickle.dump(np.array(con_vectors), f)


In [ ]:
with open(r"C:\Users\user\Desktop\Heterogeneous\MOOCCube\entities/concept.json", "r", encoding="utf8") as f:
    lines = f.readlines()
    
docs = list()
for l in lines:
    json_str = json.loads(l)
    docs.append(json_str["name"])
    if 'explanation' in json_str:
        docs.append(json_str["explanation"])

In [ ]:
import gensim

z = [list(jieba.cut(i)) for i in docs]
model = gensim.models.FastText(z, 
                               sg=0, # CBOW
                               min_n=5,
                               max_n=10,
                               vector_size=100, # change this line
                               window=5,
                               negative=10,
                               min_count=1)


In [ ]:
vocab_keys = list(model.wv.key_to_index.keys())

with open('vocab', 'w', encoding="utf8") as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL, delimiter="\n")
    lines = [l.replace("\n"," ") for l in vocab_keys]
    wr.writerow(lines)

np.savetxt("emb.tsv", model.wv.vectors, delimiter="\t")

NameError: name 'model' is not defined